In [ ]:
import pdfplumber
import os
from tqdm import tqdm
import collections
import spacy
import re

In [ ]:
#PROVARE A FARE FUNZIONE DA RUNNARE IN BASE ALL'ULTIMO CAPITOLO SALVATO (come per progetto negozi)

#potenziali (cercarne altre): 

    
#il problema è che aggiorna capitolo a  [\s]1 e quindi anche eg. 1 tonnellata; 1 anno ecc..
folder='bibles/'
testi=['Unlocked Literal Bible__.pdf', #OK
        'Easy-to-Read Version__American.pdf',  #problema titoli (dati capitolo C e verso v, e rispetto al capitolo seguente, il capitolo rimane i-1 ma v<<v-1 )
       "Young's Literal Translation__archaic British.pdf",
      "JPS TaNaKH 1917__American.pdf", #(jewish bible)
       "Revised Version with Apocrypha (1895)__archaic British",
       "Noah Webster Bible__archaic American",
       "King James (Authorized) Version__archaic British",
       "Open English Bible (Commonwealth Spelling)__",
       "Open English Bible (U. S. spelling)__",
       "Targum Onkelos Etheridge__"
       
      ]
file_=

In [ ]:
import pandas as pd
def write_csv(dict_,filename,chapter):
    keys_=[]
    values_=[]
    for item in dict_.items():
        for val in item[1]:
            for verse in val:
                keys_.append(item[0])
                values_.append(verse)
    print(keys_,values_)
    df=pd.DataFrame(columns=['Chapter','Verses'])
    df['Chapter']=keys_
    df['Verses']=values_
    df.to_csv('csv bibles/'+filename.replace('.pdf','')+';'+chapter+'.csv',index=False,header=True)
    return df
#write_csv(temp_chapter_dict,'file','Genesis')

In [ ]:
#NON ELIMINA NOTE A PIE DI PAGINA E TITOLI DI SEZIONI(rintracciabili da anomalie nell'indice <cap:verso>)
#FUNZIONA CON PDF SENZA COLONNE (nè nella prefazione, nè nel resto)
#FUNZIONA CON PDF CON 1:1 AL PRIMO CAPITOLO (skippa prefazione)

for f,file in enumerate(os.listdir(folder)): #eliminare ciclo for
    temp_chapter_dict={}
    pdf = pdfplumber.open(f'bibles/{file_}')
    index_=""""""
    idxs_pages=[]
    roman_nums=['i','ii','iii','v','vi','vii','viii','ix','x']
    target_rem=0
    for i,page in enumerate(tqdm(pdf.pages)):
        if 'content' in page.extract_text(x_tolerance=1) or ' . . . . ' in page.extract_text(x_tolerance=1):
            index_+=page.extract_text(x_tolerance=1)
            idxs_pages.append(i)
        elif '1:1' in page.extract_text(x_tolerance=1):
            target_rem=i
            break
        #any([x in 'a' for x in roman_nums ])
        #if i>=10:
            #break
    index_='\n'.join([x for x in index_.split('\n') if '.' in x])
    print(index_)
    index_=index_.split('\n')
    print(index_)
    #index_=[ x.split('.')[:-1] + ' ' + x.split('.')[-1].strip() for x in index_]
    #print('SORTED',sorted(index_,key= lambda x:int(x.strip().split(' ')[-1])))
    index_=[''.join(x.split('.')).strip() for x in index_]
    index_=[ x.split('    ')[0] + '    ' + x.split('    ')[-1].strip() for x in index_]#index_=[ + ' ' + x.split('.')[-1].strip() for x in index_]
    
    pages_=pdf.pages[target_rem:] #skip preface
    page_numbers=[x for x in range(target_rem,len(pages_)+1)]
    cap=0
    #already_seen_idx=[] #check if next chap to reset curr_cap
    already_seen_idx=[x.split(';')[-1].replace('.csv','') for x in os.listdir('csv bibles/')]
    check_list=[x for x in os.listdir('csv bibles/')]
    #for i,idx in enumerate(tqdm(clean_idx)):
        #clean_idx=' '.join(index_[i].split(' ')[:-1]).strip()
        #if
    #print('PAGE NUMBERS',page_numbers)
    for j,p in enumerate(pages_):
                page_=pages_[j].extract_text(x_tolerance=1)
                page_chapter=re.match(r'([\d]?[\s]?[A-z\s]+[\D])',page_) #([\d]?[\s]?[A-z]+[\D]) #OLD, NON PRENDE TITOLI CON SPAZI
                clean_idx=page_[page_chapter.start():page_chapter.end()].strip()
                if any([clean_idx.strip() in x for x in check_list]):
                    print('SKIPPING',clean_idx)
                    continue
                if already_seen_idx==[]: #to not launch error on [-1]
                    already_seen_idx.append(clean_idx)
                if clean_idx != already_seen_idx[-1]: #or j==len(pages_)-1
                    cap=0
                    file_name=str(already_seen_idx.index(already_seen_idx[-1])+1)+'-'+file_
                    #write_csv(target_idx=already_seen_idx[-1], new_data=temp_chapter_dict) #try to free memory at each chapter
                    write_csv(temp_chapter_dict,file_name,already_seen_idx[-1])
                    del temp_chapter_dict
                    temp_chapter_dict={}
                    
                if clean_idx not in already_seen_idx:
                    already_seen_idx.append(clean_idx)
                    
                print('ALREADY SEEN', already_seen_idx)
                #remove bottom page
                flag_no_num_line=False
                if re.search(r'([\s][\d]+:[\d]+)','\n'.join(page_.split('\n')[1:])):
                    print('FOUND')
                    temp_page_='\n'.join(page_.split('\n')[1:])
                    try:
                        temp_page_=temp_page_[:re.search(r'([\s][\d]+:[\d]+)',temp_page_).span()[0]] #remove elements at the end having <symbol\s\d:                
                        page_=temp_page_
                        flag_no_num_line=True
                    except Exception as e:
                        print('errore ',j,e)
                print('PAGE',page_)
                print('P Chap',clean_idx)
                nlp=spacy.load('en_core_web_sm')
                nlp.max_length=1500000
                #limit=len(text_)/10
                nlp.add_pipe('sentencizer')
                doc=nlp(page_,disable = ['ner', 'parser'])
                #tokens_=[x.text for x in doc]
                
                if flag_no_num_line==False:
                    text_for_chapt='\n'.join(doc.text.split('\n')[1:])
                    text_='\n'.join(doc.text.split('\n')[1:])
                elif flag_no_num_line==True:
                    text_for_chapt='\n'.join(doc.text.split('\n'))
                    text_='\n'.join(doc.text.split('\n'))

                matches_=[]
                caps_=[]
                prev_page=pages_[j].page_number
                no_number_line=re.match('(^[\D]+)',text_)
                no_number_line_res=''
                #check if initial line without number

                if no_number_line:
                    no_number_line_res+=text_[no_number_line.start():no_number_line.end()]
                    print('NO NUMBER', no_number_line_res)
                    try:
                        temp_chapter_dict[clean_idx]
                    except:
                        temp_chapter_dict[clean_idx]=[['']]
                    #if len(temp_chapter_dict[clean_idx])>0:
                    try:
                        temp_chapter_dict[clean_idx][-1][-1]+=' '+no_number_line_res.replace('\n','')
                    except:
                        temp_chapter_dict[clean_idx].append(' '+no_number_line_res.replace('\n',''))

                match_chapters=re.finditer('(^[\d]+[\\n])',text_)
                print('MATCH CHAP LIST',match_chapters)
                #temp_curr_cap=0 #assegna capitolo precedente a ultima riga capitolo ch
                for m_,match in enumerate(re.finditer('([\d]+[\\n])',text_)):
                    print('CHAP',match)
                    if text_[match.start():match.end()].endswith('\n')\
                      and text_[match.start():match.end()][0].isnumeric():
                      #and m_!=0:
                        curr_cap=text_[match.start():match.end()].replace('\n','') #chapter line
                        
                        print('IN',curr_cap)
                        
                match_lines=[x for x in re.finditer(r'(?<!:)([\d]+-?[^:])[\s]?[A-Za-z"“”]',text_)]
                for m_,match in enumerate(re.finditer(r'(?<!:)([\d]+-?[^:])[\s]?[A-Za-z"“”]',text_)): #old pattern ([\\n]?[\d]{1,}?)[^:]                  
                    matches_.append(match.span())
                    target_=text_[match.start():match.end()]
                    
                    #if int(target_.strip().split(' ')[0])>1 and curr_cap!=1\
                    #and int(next_target_.strip().split(' ')[0])==1:#
                    #    curr_cap=str(int(curr_cap)-1)
                    #    caps_.append(int(curr_cap.strip()))
                    #else:
                    #    caps_.append(curr_cap)
                clean_lines=[]
                for i,each in enumerate(matches_):
                    if i+1<len(matches_):
                        #print(text_[each[0]:matches_[i+1][0]])
                        target_=text_[each[0]:matches_[i+1][0]]
                        if re.match('(^[1][\D][^:])\D',target_):
                            cap+=1
                        print(target_)
                        clean_lines.append(str(cap).replace('\n','')+':'+target_.replace('\n',' '))
                    else: #last line of the page
                        print('LAST LINE',text_[each[0]:])
                        target_=text_[each[0]:]
                        if re.match('(^[1][\D][^:])\D',target_):
                            cap+=1
                        #                      caps_[i]
                        print(target_)
                        clean_lines.append(str(cap).replace('\n','')+':'+target_.replace('\n',' '))
                try:
                    temp_chapter_dict[clean_idx].append(clean_lines)
                except:
                    temp_chapter_dict[clean_idx]=[]
                    temp_chapter_dict[clean_idx].append(clean_lines)
                
    #if j==len(pages_)-1: #save if end of the last page
    cap=0
    file_name=str(already_seen_idx.index(clean_idx)+1)+'-'+file_
    #write_csv(target_idx=already_seen_idx[-1], new_data=temp_chapter_dict) #try to free memory at each chapter
    write_csv(temp_chapter_dict,file_name,clean_idx)
    del temp_chapter_dict
    temp_chapter_dict={}
    
    break